# Day 5 - Part 1: 모멘텀 전략 완성

## 학습 목표
- 30개 종목으로 확장하기
- 수수료별 성과 비교하기 (0%, 0.1%, 0.25%)
- 결과를 그래프로 시각화하기

## 1. 라이브러리 임포트

In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 2. 코스피 200 상위 30개 종목 선택

In [11]:
# 실습용 30개 종목 (시가총액 상위)
tickers = {
    # 반도체/IT
    '005930.KS': '삼성전자', '000660.KS': 'SK하이닉스', '035420.KS': 'NAVER',
    '035720.KS': '카카오', '051910.KS': 'LG화학', '006400.KS': '삼성SDI',
    
    # 자동차
    '005380.KS': '현대차', '012330.KS': '현대모비스', '000270.KS': '기아',
    
    # 금융
    '055550.KS': '신한지주', '086790.KS': '하나금융지주', '105560.KS': 'KB금융',
    '316140.KS': '우리금융지주',
    
    # 바이오/헬스케어
    '068270.KS': '셀트리온', '207940.KS': '삼성바이오로직스', '326030.KS': 'SK바이오팜',
    '028300.KS': 'HLB',
    
    # 유통/소비재
    '051900.KS': 'LG생활건강', '097950.KS': 'CJ제일제당', '271560.KS': '오리온',
    
    # 에너지/화학
    '009830.KS': '한화솔루션', '010950.KS': 'S-Oil', '011170.KS': '롯데케미칼',
    
    # 통신/미디어
    '017670.KS': 'SK텔레콤', '030200.KS': 'KT', '032640.KS': 'LG유플러스',
    
    # 제조/건설
    '028260.KS': '삼성물산', '009150.KS': '삼성전기', '034730.KS': 'SK',
    '000810.KS': '삼성화재'
}

print(f"총 {len(tickers)}개 종목 선택\n")

# 업종별 분포
sectors = {
    '반도체/IT': ['005930.KS', '000660.KS', '035420.KS', '035720.KS', '051910.KS', '006400.KS'],
    '자동차': ['005380.KS', '012330.KS', '000270.KS'],
    '금융': ['055550.KS', '086790.KS', '105560.KS', '316140.KS'],
    '바이오': ['068270.KS', '207940.KS', '326030.KS', '028300.KS'],
}

used_tickers = {t for lst in sectors.values() for t in lst} # set comprehension

sectors['기타'] = list(set(tickers.keys()) - used_tickers)

for sector, ticker_list in sectors.items():
    print(f"{sector}: {len(ticker_list)}개")

총 30개 종목 선택

반도체/IT: 6개
자동차: 3개
금융: 4개
바이오: 4개
기타: 13개


## 3. 데이터 다운로드

In [12]:
start_date = '2019-01-01'  # 12개월 수익률 계산을 위해
end_date = '2024-12-31'

all_data = {}
failed_tickers = []

print("데이터 다운로드 중...\n")

for ticker, name in tickers.items():
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, multi_level_index=False)
        if not data.empty and len(data) > 300:  # 최소 데이터 요건
            all_data[ticker] = data
            print(f"{name:15s}: {len(data):4d}일")
        else:
            failed_tickers.append((ticker, name))
            print(f"{name:15s}: 데이터 부족")
    except Exception as e:
        failed_tickers.append((ticker, name))
        print(f"{name:15s}: 오류")

print(f"\n{'='*60}")
print(f"성공: {len(all_data)}개 종목")
if failed_tickers:
    print(f"실패: {len(failed_tickers)}개 종목")
print(f"{'='*60}\n")

데이터 다운로드 중...

삼성전자           : 1475일
SK하이닉스         : 1475일
NAVER          : 1475일
카카오            : 1475일
LG화학           : 1475일
삼성SDI          : 1475일
현대차            : 1475일
현대모비스          : 1475일
기아             : 1475일
신한지주           : 1475일
하나금융지주         : 1475일
KB금융           : 1475일
우리금융지주         : 1442일
셀트리온           : 1475일
삼성바이오로직스       : 1475일
SK바이오팜         : 1105일
HLB            : 1475일
LG생활건강         : 1475일
CJ제일제당         : 1475일
오리온            : 1475일
한화솔루션          : 1475일
S-Oil          : 1475일
롯데케미칼          : 1475일
SK텔레콤          : 1475일
KT             : 1475일
LG유플러스         : 1475일
삼성물산           : 1475일
삼성전기           : 1475일
SK             : 1475일
삼성화재           : 1475일

성공: 30개 종목



## 4. 모멘텀 전략 클래스 (개선 버전)

In [ ]:
class MomentumStrategyV2(bt.Strategy):
    """
    개선된 모멘텀 전략
    - 매월 말: 12개월 수익률 상위 N개 선택
    - 동일 가중 투자
    - 상세 로그 출력
    """
    
    params = (
        ('rebalance_dates', None),
        ('top_n', 5),
        ('lookback', 252),  # 12개월 = 252영업일
        ('verbose', True),  # 로그 출력 여부
    )
    
    def __init__(self):
        # 12개월 수익률 계산
        self.returns = {}
        for d in self.datas:
            self.returns[d._name] = (d.close / d.close(-self.params.lookback) - 1) * 100
        
        self.rebalance_dates = [pd.Timestamp(d) for d in self.params.rebalance_dates]
        self.next_rebalance_idx = 0
        self.rebalance_count = 0
        
        if self.params.verbose:
            print(f"전략 초기화: {len(self.datas)}개 종목, 상위 {self.params.top_n}개 선택")
    
    def next(self):
        current_date = pd.Timestamp(self.datas[0].datetime.date(0))
        
        if self.next_rebalance_idx < len(self.rebalance_dates):
            next_rebalance = self.rebalance_dates[self.next_rebalance_idx]
            
            if current_date >= next_rebalance:
                self.rebalance()
                self.next_rebalance_idx += 1
    
    def rebalance(self):
        """리밸런싱 실행"""
        current_date = self.datas[0].datetime.date(0)
        self.rebalance_count += 1
        
        # Step 1: 수익률 계산 및 순위
        ranking = {}
        for d in self.datas:
            try:
                ret = self.returns[d._name][0]
                if not np.isnan(ret) and d.close[0] > 0:
                    ranking[d._name] = (d, ret)
            except:
                continue
        
        # Step 2: 상위 N개 선택
        sorted_stocks = sorted(ranking.items(), key=lambda x: x[1][1], reverse=True)
        top_n = [item[1][0] for item in sorted_stocks[:self.params.top_n]]
        
        # Step 3: 기존 포지션 정리
        closed_positions = []
        for d in self.datas:
            if self.getposition(d).size > 0:
                if d not in top_n:
                    self.close(d)
                    closed_positions.append(d._name)
        
        # Step 4: 동일 가중 투자
        target_value = self.broker.getvalue() / self.params.top_n
        new_positions = []
        
        for d in top_n:
            current_value = self.getposition(d).size * d.close[0]
            target_shares = int(target_value / d.close[0])
            diff = target_shares - self.getposition(d).size
            
            if abs(diff) > 0:
                if diff > 0:
                    self.buy(data=d, size=diff)
                else:
                    self.sell(data=d, size=-diff)
                
                if self.getposition(d).size == 0:
                    new_positions.append(d._name)
        
        # 로그 출력
        if self.params.verbose and self.rebalance_count % 3 == 1:  # 3개월마다만 출력
            print(f"\n{'='*60}")
            print(f"{current_date} (리밸런싱 #{self.rebalance_count})")
            print(f"포트폴리오 가치: {self.broker.getvalue():,.0f}원")
            print(f"\n선택된 종목 (상위 {self.params.top_n}개):")
            for i, d in enumerate(top_n, 1):
                ret = ranking[d._name][1]
                print(f"  {i}. {d._name:12s}: {ret:7.2f}%")
            print(f"{'='*60}")

## 5. 리밸런싱 날짜 준비

In [ ]:
def get_month_end_dates(data, start, end):
    """매월 말 거래일 찾기"""
    mask = (data.index >= start) & (data.index <= end)
    filtered = data[mask]
    month_ends = filtered.groupby(filtered.index.to_period('M')).apply(
        lambda x: x.index[-1]
    ).tolist()
    return month_ends

# 첫 번째 종목으로 날짜 생성
first_ticker = list(all_data.keys())[0]
rebalance_dates = get_month_end_dates(
    all_data[first_ticker],
    start='2020-01-01',
    end='2023-12-31'
)

print(f"{len(rebalance_dates)}번의 리밸런싱 예정")
print(f"   첫 리밸런싱: {rebalance_dates[0].strftime('%Y-%m-%d')}")
print(f"   마지막 리밸런싱: {rebalance_dates[-1].strftime('%Y-%m-%d')}")

## 6. 백테스트 함수 (수수료별 비교용)

In [ ]:
def run_backtest(data_dict, commission=0.001, top_n=5, verbose=False):
    """
    백테스트 실행 함수
    
    Parameters:
    - data_dict: 종목별 데이터
    - commission: 수수료 (0.001 = 0.1%)
    - top_n: 선택할 종목 수
    - verbose: 로그 출력 여부
    """
    cerebro = bt.Cerebro()
    
    # 설정
    cerebro.broker.setcash(100_000_000)
    cerebro.broker.setcommission(commission=commission)
    
    # 데이터 추가
    for ticker, data in data_dict.items():
        data_bt = bt.feeds.PandasData(
            dataname=data,
            name=ticker,
            fromdate=datetime(2020, 1, 1),
            todate=datetime(2023, 12, 31)
        )
        cerebro.adddata(data_bt)
    
    # 전략 추가
    cerebro.addstrategy(
        MomentumStrategyV2,
        rebalance_dates=rebalance_dates,
        top_n=top_n,
        verbose=verbose
    )
    
    # 분석기 추가
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.02)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    
    # 실행
    start_value = cerebro.broker.getvalue()
    results = cerebro.run()
    end_value = cerebro.broker.getvalue()
    
    # 결과 추출
    strat = results[0]
    sharpe = strat.analyzers.sharpe.get_analysis()
    dd = strat.analyzers.drawdown.get_analysis()
    returns = strat.analyzers.returns.get_analysis()
    trades = strat.analyzers.trades.get_analysis()
    
    return {
        'cerebro': cerebro,
        'start_value': start_value,
        'end_value': end_value,
        'profit': end_value - start_value,
        'return_pct': (end_value - start_value) / start_value * 100,
        'annual_return': returns.get('rnorm', 0),
        'sharpe': sharpe.get('sharperatio', 0) if sharpe.get('sharperatio') is not None else 0,
        'max_dd': dd['max']['drawdown'],
        'trades': trades
    }

## 7. 수수료별 비교 백테스트

In [ ]:
# 테스트할 수수료 시나리오
commission_scenarios = [
    (0.0000, '수수료 0%'),
    (0.0010, '수수료 0.1%'),
    (0.0025, '수수료 0.25%')
]

results = {}

print(f"\n{'='*70}")
print(f"모멘텀 전략 백테스트 시작 (30개 종목)")
print(f"{'='*70}\n")

for commission, label in commission_scenarios:
    print(f"\n{'─'*70}")
    print(f"{label} 백테스트 중...")
    print(f"{'─'*70}")
    
    result = run_backtest(
        all_data,
        commission=commission,
        top_n=5,
        verbose=(commission == 0.0010)  # 0.1%만 상세 로그
    )
    
    results[label] = result
    
    print(f"\n{label} 완료!")
    print(f"   최종 자금: {result['end_value']:,.0f}원")
    print(f"   수익률: {result['return_pct']:.2f}%")

## 8. 결과 비교표

In [ ]:
print(f"\n{'='*90}")
print(f"수수료별 성과 비교")
print(f"{'='*90}")
print(f"{'항목':<20} {'0%':>20} {'0.1%':>20} {'0.25%':>20}")
print(f"{'─'*90}")

metrics = [
    ('총 수익률 (%)', 'return_pct', '{:.2f}%'),
    ('연간 수익률 (%)', 'annual_return', '{:.2f}%'),
    ('최종 자금 (만원)', 'end_value', '{:,.0f}'),
    ('수익금 (만원)', 'profit', '{:,.0f}'),
    ('샤프 비율', 'sharpe', '{:.3f}'),
    ('최대 낙폭 (%)', 'max_dd', '{:.2f}%'),
]

for metric_name, key, fmt in metrics:
    row = [metric_name]
    for _, label in commission_scenarios:
        value = results[label][key]
        if key in ['end_value', 'profit']:
            value = value / 10000  # 만원 단위
        row.append(fmt.format(value))
    print(f"{row[0]:<20} {row[1]:>20} {row[2]:>20} {row[3]:>20}")

print(f"{'='*90}\n")

## 9. 수수료 영향 분석

In [ ]:
# 수수료로 인한 손실 계산
base_profit = results['수수료 0%']['profit']
loss_01 = base_profit - results['수수료 0.1%']['profit']
loss_025 = base_profit - results['수수료 0.25%']['profit']

print(f"수수료 영향 분석\n")
print(f"수수료 0.1%로 인한 손실:")
print(f"  - 금액: {loss_01:,.0f}원")
print(f"  - 비율: {loss_01/base_profit*100:.2f}%")
print(f"\n수수료 0.25%로 인한 손실:")
print(f"  - 금액: {loss_025:,.0f}원")
print(f"  - 비율: {loss_025/base_profit*100:.2f}%")

## 10. 결과 시각화

In [ ]:
# 포트폴리오 가치 그래프 (0.1% 수수료)
print("\n포트폴리오 가치 그래프 생성 중...")
results['수수료 0.1%']['cerebro'].plot(style='candlestick', barup='red', bardown='blue')
plt.show()

## 11. 수익률 비교 차트

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 총 수익률 비교
labels = [label for _, label in commission_scenarios]
returns = [results[label]['return_pct'] for label in labels]
colors = ['green', 'orange', 'red']

axes[0].bar(labels, returns, color=colors, alpha=0.7)
axes[0].set_title('총 수익률 비교', fontsize=14, fontweight='bold')
axes[0].set_ylabel('수익률 (%)', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(returns):
    axes[0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

# 샤프 비율 비교
sharpes = [results[label]['sharpe'] for label in labels]

axes[1].bar(labels, sharpes, color=colors, alpha=0.7)
axes[1].set_title('샤프 비율 비교', fontsize=14, fontweight='bold')
axes[1].set_ylabel('샤프 비율', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
for i, v in enumerate(sharpes):
    axes[1].text(i, v + 0.05, f'{v:.2f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 12. 핵심 인사이트

**배운 점**:

1. **수수료의 중요성**
   - 0.1%에서 0.25%로 증가하면 수익률이 크게 감소
   - 리밸런싱 횟수가 많을수록 수수료 영향 증가

2. **종목 수의 영향**
   - 많은 종목 = 분산 효과 증가
   - 적은 종목 = 집중 투자 효과

3. **현실적 가정**
   - 실제 거래에서는 슬리피지도 고려해야 함
   - 시장 충격 비용도 존재

**주의사항**:
- 과거 성과 ≠ 미래 성과
- 시장 환경 변화를 항상 고려
- 과적합(overfitting) 주의

## 정리

완료한 작업:
- 30개 종목으로 확장
- 수수료별 성과 비교
- 결과 시각화 및 분석